# NYC Bike Rental Analysis

TBD - Introduction paragraph
- Set the context
- Introduce the analysis
- Add an image?


## 1. A story in 9+ million rows

TBD - Introduce the data set
- Where does it come from
- What were some of the challenges - # of rows and optimization
- What are the steps in this section



### Import packages


In [1]:
# Import packages
import glob
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='ticks', color_codes=True, font_scale=1.25)

# Set display option for floats in Pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)


### Create helper functions


In [2]:
# Function for reading in and processing data files
def read_and_process(filepath, col_types, drop_cols, col_names, get_stations=False):
    '''Reads in and processes CitiBike monthly csv data files and returns a 
    dataframe containing the list of rentals for a given month
    '''
    # Read in the data
    temp_df = pd.read_csv(filepath) 
    
    # Drop rows with null values
    temp_df = temp_df.dropna()
    
    # Convert start/stop time columns to datetime
    temp_df.starttime = pd.to_datetime(temp_df.starttime, infer_datetime_format=True)
    temp_df.stoptime = pd.to_datetime(temp_df.stoptime, infer_datetime_format=True)
    
    # Convert column dtypes
    temp_df = temp_df.astype(col_types)
    
    if get_stations == True:  # Process station data
        temp_stations = process_station_data(temp_df)  
    
    # Drop unnecessary or redundant columns
    temp_df = temp_df.drop(drop_cols, axis=1)
    
    # Rename remaining columns
    temp_df.columns = col_names
    
    # Return df object(s) based on params
    if get_stations == True: 
        return temp_df, temp_stations
    else:
        return temp_df


# Function for abstracting out station-related data
def process_station_data(df):
    '''Processes and abstracts station-related data and returns as a separate dataframe
    '''
    # New column names
    cols = ['id', 'name', 'lat', 'lon']

    # Temp df for start stations
    start_stations = df[['start station id', 'start station name',
                         'start station latitude', 'start station longitude']]
    start_stations.columns = cols
    start_stations = start_stations.drop_duplicates()

    # Temp df for end stations
    end_stations = df[['end station id', 'end station name',
                       'end station latitude', 'end station longitude']]
    end_stations.columns = cols
    end_stations = end_stations.drop_duplicates()
    
    # Concatenate the start/end station dfs and drop dups 
    temp_stations = pd.concat([start_stations, end_stations],
                              ignore_index=True).drop_duplicates()
    
    return temp_stations


# Function for pivoting dataframe
def pivot_df(df, pv_index, pv_columns, pv_values, scale, reset=False, cumsum=False, impute=False):
    """Returns a pivoted version of a dataframe to aid in plotting
    """
    # Create pivot table
    df_pivot = df.pivot_table(index=pv_index, columns=pv_columns, values=pv_values, aggfunc=np.sum)

    # Impute values for 2019-02-29 to account for leap year in 2020 
    if impute == True:
        df_pivot = df_pivot.fillna(df_pivot[2019].mean())

    # Convert to running total
    if cumsum == True:
        df_pivot = df_pivot.cumsum()
    
    # Scale values 
    df_pivot = df_pivot.apply(lambda x: round((x * scale),2), axis=1)

    # Reset the index
    if reset == True:
        df_pivot = df_pivot.reset_index()
    
    return df_pivot


### Prep for import and read in the data


In [ ]:
# Dictionary of columns and optimal dtypes
col_types = {'usertype': 'category', 'birth year': 'int', 'gender': 'int8'}

# Create list of updated column names
col_names = ['tripduration', 'starttime', 'usertype', 'birth_year', 'gender']

# Create list of columns to drop
drop_cols = ['stoptime', 'start station id', 'end station id', 'start station name',
             'start station latitude', 'start station longitude', 'end station name',
             'end station latitude', 'end station longitude', 'bikeid']

# Read in and process the data
data_files = glob.glob('../data/' + "*.csv")
rentals_dfs = []
for file in data_files:
    rentals_df = read_and_process(file, col_types, drop_cols, col_names)
    rentals_dfs.append(rentals_df)

# Concatentate the list of temp dfs
df = pd.concat(rentals_dfs, ignore_index=True)

# Calculate memory usage
mem_usage = df.memory_usage().sum() / 1024**2

# Print output
print("Dataframe rows, columns: " + str(df.shape))
print('Memory usage after optimization:  {:.2f} MB'.format(mem_usage))


## 2. How does a global pandemic affect bike rentals?

TBD - Intro the section

- Add more date dimensions
- Add a count column
- Trip count by dow, hod (gender?, age? usertype?)

### Create new columns for rental count and day/month/year

In [ ]:
# Sort the dfs by starttime
df = df.sort_values(by=['starttime']).reset_index(drop=True)

# Create a column to facilitate totaling rental count
df['rental_count'] = 1

# Create new columns for day, month and year
df['hour'] = df.starttime.dt.hour
df['day'] = df.starttime.dt.day
df['month'] = df.starttime.dt.month
df["year"] = df.starttime.dt.year

# Drop starttime column
df = df.drop('starttime', axis=1)


### Pivot by total rentals per month, by year


In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month'], ['year'], 'rental_count', 0.000001)

# Rename index to friendly month names
df_pivot.rename(index={1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr'}, inplace=True)

# Plot the data and set style
labels = ['Jan', 'Feb', 'Mar', 'Apr']
df_pivot.plot(kind='bar', figsize=(12, 7), rot=0, label=labels, color=['gray', 'k'])
plt.text(0.1, .91, 'Millions', fontsize=14, transform=plt.gcf().transFigure)
plt.xlabel(None)
plt.suptitle("Rentals Per Month", fontsize=18)
# plt.title("(CitiBike NYC)", fontsize=14, pad=16)
plt.legend(title=None, loc='upper left', bbox_to_anchor=(1, 1));


### Pivot by cumulative sum of rentals, by year


In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'day'], ['year'], 'rental_count', 1.00E-6, cumsum=True, impute=True)

# Plot a line chart and set the chart style
df_pivot.plot(kind='line', linewidth=3, figsize=(12, 7), color=['gray', 'k'])

# Plot vertical lines for key dates
plt.axvline(x=70, color='red', linestyle='--', linewidth=1)
plt.text(67.5,0.05,'Covid declared pandemic',rotation=90, fontsize=12, color='red')
plt.axvline(x=80, color='red', linestyle='--', linewidth=1)
plt.text(77.5,0.05,'NY shelter-in-place order',rotation=90, fontsize=12, color='red')

# Add labels and legend
plt.text(0.1, .91, 'Millions', fontsize=14, transform=plt.gcf().transFigure)
plt.suptitle("Total Cumulative Rental Count", fontsize=18)
# plt.title("(millions of rentals)", fontsize=14, pad=16)
plt.xticks(np.linspace(0, 121, num=5), ['Jan', 'Feb', 'Mar', 'Apr', 'May'])
plt.xlabel(None)
plt.legend(title=None, loc='upper left', bbox_to_anchor=(1, 1));


## 3. What's the affect on rental durations?


In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'day'], ['year'], 'tripduration', 1.667E-8, cumsum=True, impute=True)

# Plot a line chart and set the chart style
df_pivot.plot(kind='line', linewidth=3, figsize=(12, 7), color=['gray', 'k'])

# Plot vertical lines for key dates
plt.axvline(x=70, color='red', linestyle='--', linewidth=1)
plt.text(67.5,0.05,'Covid declared pandemic',rotation=90, fontsize=12, color='red')
plt.axvline(x=80, color='red', linestyle='--', linewidth=1)
plt.text(77.5,0.05,'NY shelter-in-place order',rotation=90, fontsize=12, color='red')

# Add labels and legend
plt.text(0.1, .91, 'Millions', fontsize=14, transform=plt.gcf().transFigure)
plt.suptitle("Total Cumulative Rental Duration", fontsize=18)
plt.title("(in minutes)", fontsize=14, pad=16)
plt.xticks(np.linspace(0, 121, num=5), ['Jan', 'Feb', 'Mar', 'Apr', 'May'])
plt.xlabel(None)
plt.legend(title=None, loc='upper left', bbox_to_anchor=(1, 1));


## 4. Do age or gender make a difference?

TBD intro
- Review rental counts


### Create helper function for plotting


In [ ]:
# Create helper function for age & gender plots
def make_plot(df, x_labels, num_plots, facet, y_min, y_max, subtitle, title, x_pos_suptitle, x_pos_title):
    """Returns a plot based on a pivoted df
    """
    # Set x labels
    x_labels = x_labels

    # Create figure and pad the subplots
    fig = plt.figure(figsize=(16,4))
    # fig.subplots_adjust(wspace=0.3) # Use to adjust spacing on plots if needed

    # Create a subplot for each gender
    for i in range(num_plots):
        df_temp = df_pivot[df_pivot[facet] == i]
        df_temp_pivot = df_temp.pivot_table(index=['month'], values=[2019,2020], aggfunc=np.sum)

        # Extract x and y values 
        x = df_temp_pivot.index.to_list()
        y1 = df_temp_pivot.iloc[:,0].to_list()
        y2 = df_temp_pivot.iloc[:,1].to_list()
        
        # Create elements to zip y values
        y = [y1, y2]
        labels = ['2019', '2020']
        colors = ['gray', 'k']

        # Plot the values
        plt.subplot(1,3, (i+1))
        for y_arr, label, color in zip(y, labels, colors):
            plt.plot(x, y_arr, color=color, label=label, linewidth=2, marker='o')
        plt.xticks(np.linspace(1, 4, 4), ['Jan', 'Feb', 'Mar', 'Apr'])
        plt.xlabel(x_labels[i], labelpad=16)
#         if i !=0:                                  # An option to remove yticklabels on inner plots
#             plt.yticks(np.linspace(-0, 5, 5), " ") # Will require abstraction of spacing if implemented
        plt.ylim(y_min, y_max) 
        
    # Add legend and show the plot
    plt.text(0.1, .94, 'Millions', fontsize=14, transform=plt.gcf().transFigure)
    plt.suptitle(subtitle, y=1.05, x=x_pos_suptitle, fontsize=18)
    plt.title(title, fontsize=14, pad=16, y=1.0, x=x_pos_title)
    plt.legend(title=None, loc='upper left', bbox_to_anchor=(1, 1));
    plt.show()     


### Plot cumulative rental count and duration by gender


In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'gender'], ['year'], 'rental_count', 1.00E-6,
                    cumsum=True, impute=True, reset=True)

# Configure variables
x_labels = ['Not Specified', 'Male', 'Female']
num_plots = 3
facet = 'gender'
y_min, y_max = -0.5, 5.5
subtitle = "Total Cumulative Rentals, by Gender"
title = "(millions of rentals)"
x_pos_suptitle, x_pos_title = 0.5, -0.75

# Call the helper function to make the plot
make_plot(df, x_labels, num_plots, facet, y_min, y_max, subtitle, title, x_pos_suptitle, x_pos_title)

In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'gender'], ['year'], 'tripduration', 1.667E-8,
                    cumsum=True, impute=True, reset=True)

# Update variables
y_min, y_max = -0.5, 80
subtitle = "Total Cumulative Duration, by Gender"
title = "(minutes)"

# Call the helper function to make the plot
make_plot(df, x_labels, num_plots, facet, y_min, y_max, subtitle, title, x_pos_suptitle, x_pos_title)


### Assign age groups


In [ ]:
# Create function to return decade for a given year
def assign_age_cat(year):
    """Returns 1 digit age category when passed 4 digit year    
    """
    if (year > 1985) and (year <= 2005):
        age_cat = 0
    elif (year >= 1970) and (year <= 1985):
        age_cat = 1
    elif (year >= 1920) and (year < 1970):
        age_cat = 2
    else:
        age_cat = 3
    return age_cat

In [ ]:
# Create new column for decade and populate with function
df["age_cat"] = df['birth_year'].apply(assign_age_cat)

# Drop rows with no birth year
df = df[df.age_cat != 3]


### Plot cumulative rental count and duration by age group


In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'age_cat'], ['year'], 'rental_count', 1.00E-6,
                    cumsum=True, impute=True, reset=True)

# Update variables
x_labels = ['Under 35', '35-55', 'Over 55']
facet = 'age_cat'
y_min, y_max = -0.5, 5.5
subtitle = "Total Cumulative Rentals, by Age Group"
title = "(millions of rentals)"
x_pos_suptitle, x_pos_title = 0.5, -0.85

# Call the helper function to make the plot
make_plot(df, x_labels, num_plots, facet, y_min, y_max, subtitle, title, x_pos_suptitle, x_pos_title)

In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'age_cat'], ['year'], 'tripduration', 1.667E-8,
                    cumsum=True, impute=True, reset=True)

# Update variables
y_min, y_max = -0.5, 80
subtitle = "Total Cumulative Duration, by Age Group"
title = "(minutes)"

# Call the helper function to make the plot
make_plot(df, x_labels, num_plots, facet, y_min, y_max, subtitle, title, x_pos_suptitle, x_pos_title)


## 4. How is this impacting the bottom line?

TBD intro


### Plot cumulative rental count and duration by user type


In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'usertype'], ['year'], 'rental_count', 1.00E-6,
                    cumsum=True, impute=True, reset=True)

# Replace usertype values with 0 and 1
dict = {'Customer': 0, 'Subscriber': 1}
df_pivot = df_pivot.replace({"usertype": dict}) 

# Update variables
x_labels = ['Casual Customer', 'Subscriber']
num_plots = 2
facet = 'usertype'
y_min, y_max = -0.5, 5.5
subtitle = "Total Cumulative Rentals, by User Type"
title = "(millions of rentals)"
x_pos_suptitle, x_pos_title = 0.35, -0.25

# Call the helper function to make the plot
make_plot(df, x_labels, num_plots, facet, y_min, y_max, subtitle, title, x_pos_suptitle, x_pos_title)

In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['month', 'usertype'], ['year'], 'tripduration', 1.667E-8,
                    cumsum=True, impute=True, reset=True)

# Replace usertype values with 0 and 1
dict = {'Customer': 0, 'Subscriber': 1}
df_pivot = df_pivot.replace({"usertype": dict}) 

# Update variables
y_min, y_max = -5, 80
subtitle = "Total Cumulative Duration, by User Type"
title = "(minutes)"

# Call the helper function to make the plot
make_plot(df, x_labels, num_plots, facet, y_min, y_max, subtitle, title, x_pos_suptitle, x_pos_title)



TBD summary




## 5. What is the impact in terms of peak bike usage, by hour of day and area?


### Plot distribution of rentals and duration by hour of day


In [ ]:
# Helper function for plotting

def plot_hr_dist(df, y_tick_loc, y_min, y_max, y_axis_label, suptitle, title):
    """Returns a plot based on a pivoted df
    """
    # Set x labels and x values
    x_labels = ['Jan', 'Feb', 'Mar', 'Apr']
    x = df_pivot.index.to_list()

    # Set colors & labels
    colors = {2019:'gray', 2020: 'k'}
    labels = {2019:'2019', 2020: '2020'}

    # Create figure and pad the subplots
    fig = plt.figure(figsize=(16,7))
    fig.subplots_adjust(wspace=0.3)

    # Create a subplot for each month, year combination
    for year in range(2019, 2021, 1):
        for i in range(4):
            # Extract y values 
            y = df_pivot[year, i+1]

            # Plot the values
            if year == 2020:
                i=i+4
            plt.subplot(2, 4, (i+1))
            plt.bar(x, y, label=labels[year], color=colors[year])
            
            # Set xticks including tick labels for bottom row
            if i > 3:
                plt.xticks(np.linspace(0, 24, 5), ['12am', '6am', '12pm', '6PM'], rotation=0, fontsize=12)
                plt.xlabel(x_labels[i-4], labelpad=16)
            else:
                plt.xticks(np.linspace(0, 24, 5), " ")

            # Set yticks including tick labels for bottom row
            if i !=0 and i !=4:
                plt.yticks(y_tick_loc, " ")
            
            # Set axis limits
            plt.ylim(y_min, y_max)
            plt.xlim(-.5,23.5)

            # Configure legend for outermost column
            if i == 3 or i == 7:
                plt.legend(title=None, loc='upper left', bbox_to_anchor=(1, 1))
    #         elif i == 7: # Use if desired to group legends 
    #             plt.legend(title=None, loc='upper left', bbox_to_anchor=(1, 2))

    # Add text & titles
    plt.text(0.1, .92, y_axis_label, fontsize=14, transform=plt.gcf().transFigure)
    plt.suptitle(suptitle, y=1, x=.5, fontsize=18)
    plt.title(title, fontsize=14, pad=16, y=2.25, x=-1.6)

    # Show the plot
    plt.show()

In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['hour'], ['year', 'month'], 'rental_count', 1.00E-3,
                    impute=True, reset=True)

# Configure variables
y_tick_loc = np.linspace(0, 200, 5)
y_min, y_max = 0, 180
y_axis_label= 'Thousands'
suptitle = "Rental Distribution by Hour of Day, Jan-Apr"
title = "(thousands of rentals)"

# Call the helper function to make the plot
plot_hr_dist(df, y_tick_loc, y_min, y_max, y_axis_label, suptitle, title)

In [ ]:
# Pivot df using helper function
df_pivot = pivot_df(df, ['hour'], ['year', 'month'], 'tripduration', 1.667E-8,
                    impute=True, reset=True)

# Update variables
y_tick_loc = np.linspace(0, 3, 6)
y_min, y_max = 0, 3
y_axis_label= 'Millions'
suptitle = "Rental Duration Distribution by Hour of Day, Jan-Apr"
title = "(minutes)"

# Call the helper function to make the plot
plot_hr_dist(df, y_tick_loc, y_min, y_max, y_axis_label, suptitle, title)

TBD - Summary


### Clear memory for new analysis


In [ ]:
del df_temp_pivot
del df_pivot
del df


### Prep for import and read in the data


In [3]:
# Dictionary of columns and optimal dtypes
col_types = {'start station id': 'int', 'end station id': 'int'}

# Create list of updated column names
col_names = ['starttime', 'stoptime', 'start_station_id', 'end_station id']

# Create list of columns to drop
drop_cols = ['gender', 'tripduration', 'birth year', 'start station name',
             'start station latitude', 'start station longitude', 'end station name',
             'end station latitude', 'end station longitude', 'bikeid', 'usertype']

# Create lists for each set of dataframes
rentals_dfs = []
station_dfs = []

# Read in and process the data
data_files = glob.glob('../data/' + "*.csv")
for file in data_files:
    rentals_df, station_df = read_and_process(file, col_types, drop_cols, col_names, get_stations=True)
    rentals_dfs.append(rentals_df)
    station_dfs.append(station_df)

# Concatenate into 2 dataframes - rentals & stations and drop duplicate stations
df = pd.concat(rentals_dfs, ignore_index=True)
stations = pd.concat(station_dfs, ignore_index=True).drop_duplicates(subset='id', keep="first")

# Calculate memory usage
rentals_mem = df.memory_usage().sum() / 1024**2
stations_mem = stations.memory_usage().sum() / 1024**2

# Print output
print("Rentals: " + str(df.shape[0]) + " rows")
print('Memory usage after optimization:  {:.2f} MB'.format(rentals_mem))
print("Stations: " + str(stations.shape[0]) + " rows")
print('Memory usage after optimization:  {:.2f} MB'.format(stations_mem))

Rentals: 9143693 rows
Memory usage after optimization:  279.04 MB
Stations: 970 rows
Memory usage after optimization:  0.04 MB


In [4]:
df.head()

,starttime,stoptime,start_station_id,end_station id
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160,3283
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519,518
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171,3154
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504,3709
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229,503


In [5]:
stations.head()

,id,name,lat,lon
0,3160,Central Park West & W 76 St,40.78,-73.97
1,519,Pershing Square North,40.75,-73.98
2,3171,Amsterdam Ave & W 82 St,40.79,-73.98
3,504,1 Ave & E 16 St,40.73,-73.98
4,229,Great Jones St,40.73,-73.99



## 5. Which areas are most impacted?


In [ ]:
import fiona
import geopandas as gpd

from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.plotting import figure, show, save
from bokeh.io import show, output_file, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper,GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import Reds6 as palette
from bokeh.resources import CDN
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon
from shapely.prepared import prep
from mpl_toolkits.basemap import Basemap   
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize
from descartes import PolygonPatch

# Setup Bokeh to output directly to the notebook
output_notebook(resources=None, verbose=False, hide_banner=True, load_timeout=5000, notebook_type='jupyter')

In [ ]:
#Turn the GeoJson file into a data frame and merge it with df 
data = '../data/nyc_zips.geojson'
gdf = gpd.read_file(data)
# merge = gdf.merge(df, how='left', on='postalCode')
gdf